<a href="https://colab.research.google.com/github/GJHSimmons/personal/blob/main/chemical_reaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook simulates a second-order chemical reaction $A+B \longrightarrow C$ using both a deterministic simulation and a stochastic simulation.

In the deterministic simulation, the reaction rate is proportional to the product of the concentrations of $A$ and $B$, denoted $[A]$ and $[B]$. The system of differential equations is

$$
\begin{align*}
\frac{d[A]}{dt} &= - k [A] [B] \\
\frac{d[B]}{dt} &= - k [A] [B] \\
\frac{d[C]}{dt} &= k [A] [B]
\end{align*}
$$

where $k$ is the base reaction rate.

In the stochastic simulation, every time two units of $A$ and $B$ interact, they combine to form a unit of $C$. The time between interactions of $A$ and $B$ is modelled using an exponential distribution with parameter $\frac{1}{k[A][B]}$.

For both simulations, the initial concentrations, reaction rate, and simulation time, can be modified in the following cell.

When you are happy, select `Runtime > Run all` from the top menu, or press `Ctrl+F9`. A plot of the simulation will appear.

In [1]:
## Parameters

# Initial concentrations
A0 = 50
B0 = 50
C0 = 0

# Reaction rate
k = 0.01

# Times
t_end = 100

In [2]:
#@title Deterministic simulation code

import numpy as np
from scipy.integrate import solve_ivp
import plotly.express as px
import pandas as pd

z0 = [A0, B0, C0]
t_start = 0

def f(t,z):
    a,b,_ = z
    return [-k*a*b, -k*a*b, k*a*b]

sol = solve_ivp(f, [t_start, t_end], z0, rtol=1e-10, atol=1e-10, dense_output=True)

t = np.linspace(t_start,t_end,10001)

df = pd.DataFrame({"t": t, "A": sol.sol(t)[0], "B": sol.sol(t)[1], "C": sol.sol(t)[2]})

fig = px.line(data_frame=df, x="t", y=["A", "B", "C"], title="Deterministic model", height=400, width=600)
fig.show()

In [3]:
#@title Stochastic simulation code

def gillespie(A0, B0, C0, k, t_end):
    t = 0
    A, B, C = A0, B0, C0
    times = [t]
    A_vals, B_vals, C_vals = [A], [B], [C]

    while t < t_end and A > 0 and B > 0:
        rate = k * A * B
        if rate == 0:
            break
        dt = np.random.exponential(1 / rate)
        t += dt
        A -= 1
        B -= 1
        C += 1
        times.append(t)
        A_vals.append(A)
        B_vals.append(B)
        C_vals.append(C)

    return np.array(times), np.array(A_vals), np.array(B_vals), np.array(C_vals)

# Run stochastic simulation
t_stoch, A_stoch, B_stoch, C_stoch = gillespie(A0, B0, C0, k, t_end)

df = pd.DataFrame({"t": t_stoch, "A": A_stoch, "B": B_stoch, "C": C_stoch})

fig = px.line(data_frame=df, x="t", y=["A", "B", "C"], title="Stochastic model", height=400, width=600)
fig.show()
